In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm, solve

In [ ]:
import numpy as np

def KKT(G,C,n,z):
    m=2*n
    N=n + 2*m
    M_KKT=np.zeros((N,N))
    
    #First row
    for i in range(0,n):
        for j in range(0,n):
            M_KKT[i,j]=G[i,j]
        for j in range(n,n+m):
            M_KKT[i,j]=-C[i,j-n]
            
    #First column
    for j in range(0,n):
        for i in range(n,n+m):
            M_KKT[i,j]=-C[j,i-n]
            
    #Bottom right matrix
    for i in range(n+m,N):
        M_KKT[i,i-m]=z[i]
        M_KKT[i-m,i]=1
        M_KKT[i,i]=z[i-m]
        
    return M_KKT

In [ ]:
#Function that computes the function F
def F(G,C,d,g,n,z):
    m=2*n
    N=n + 2*m
    #Initializations of variables
    
    F=np.zeros(N)
    x=np.zeros(n)
    lanbda=np.zeros(m)
    s=np.zeros(m)
    
    for i in range(0,n):
        x[i]=z[i]
    for i in range(0,m):
        lanbda[i]=z[i+n]
        s[i]=z[i+n+m]
    
    #First component of F
    aux=np.zeros(n)
    aux=np.matmul(G,x) + g - np.matmul(C,lanbda)
    for i in range(0,n):
        F[i]=aux[i]
    #Second component of F
    aux=np.zeros(m)
    aux=s+d-np.matmul(C.T,x)
    for i in range(n,n+m):
        F[i]=aux[i-n]
    #Last component of F
    aux=s*lanbda
    for i in range(n+m,N):
        F[i]=aux[i-n-m]
    
    return F

In [ ]:
#Function that computes the step-size correction substep(given code)
#PROBLEM C1
def Newton_step(lamb0,dlamb,s0,ds):
    alp=1
    idx_lamb0=np.array(np.where(dlamb<0))
    if idx_lamb0.size>0:
        alp = min(alp,np.min(-lamb0[idx_lamb0]/dlamb[idx_lamb0]))
    
    idx_s0=np.array(np.where(ds<0))
    if idx_s0.size>0:
        alp = min(alp,np.min(-s0[idx_s0]/ds[idx_s0]))
    
    return alp

In [ ]:
def Newton_mod(G,C,d,g,n,z0,boolean=False):
    #Varaibles we need
    m=2*n
    N=n+2*m
    lanbda=np.zeros(m)
    s=np.zeros(m)
    delta_lanbda=np.zeros(m)
    delta_s=np.zeros(m)
    e=np.ones(m)
    new_vect=np.zeros(N)
    condnumber=np.zeros(100)
    #Conditions to enter in the while
    iterations=0
    r_L=np.ones(n)
    r_C=np.ones(m)
    mu=1
    while(np.linalg.norm(r_L,2)>1e-16 and np.linalg.norm(r_C,2)>1e-16 and abs(mu)>1e-16 and iterations<=100):
        #1.Predictor substep 
        
        M_KKT=KKT(G,C,n,z0)
        delta_z=np.linalg.solve(M_KKT,-F(G,C,d,g,n,z0))
        print("F: ", -F(G,C,d,g,n,z0))
        #We compute the condition number of the matrix for every step
        if(boolean==True):
            condnumber[iterations]=np.linalg.cond(M_KKT,2)
        print("delta_z: ", delta_z)
        #2.Step-size correction substep
        
        for i in range(0,m):
            lanbda[i]=z0[i+n]
            s[i]=z0[i+n+m]
        for i in range(n,n+m):
            delta_lanbda[i-n]=delta_z[i]
            delta_s[i-n]=delta_z[i+m]
        alpha=Newton_step(lanbda,delta_lanbda,s,delta_s)
        
        print("alpha: ", alpha)
        #3.Several computations
        
        mu=np.dot(s,lanbda)/m
        mu_tilde=np.dot(s+alpha*delta_s,lanbda+alpha*delta_lanbda)/m
        sigma=(mu_tilde/mu)**3
        print("mu: ", mu)
        print("mu_tilda: ", mu_tilde)
        print("sigma: ", sigma)
        #4.Corrector substep
        new_vect=-F(G,C,d,g,n,z0)
        new_vect[n+m:N]=new_vect[n+m:N] -delta_s*delta_lanbda + sigma*mu*e
     
        delta_z=np.linalg.solve(M_KKT,new_vect)
        
        #5.Step-size correction substep
        
        for i in range(n,n+m):
            delta_lanbda[i-n]=delta_z[i]
            delta_s[i-n]=delta_z[i+m]
        alpha=Newton_step(lanbda,delta_lanbda,s,delta_s)
        print("new rhv: ", new_vect)
        print("new delta_z: ", delta_z)
        print("new alpha:", alpha)
        #6.Update substep
        
        z0=z0+0.95*alpha*delta_z
        print("updated z: ", z0)
        iterations=iterations+1
        print('end iter n: ', iterations)
        r_L=F(G,C,d,g,n,z0)[0:n]
        r_C=F(G,C,d,g,n,z0)[n:n+m]
    return z0,iterations,condnumber

In [ ]:
# Defining a function to generate the test problem for a size n ----------------
def test_prob_gen(n=2):
  # Defining the test problem ----------------------------------------------------
  m = 2*n

  G = np.eye(n)
  g = np.random.normal(0, 1, n)
  C = np.block([ [np.eye(n), -np.eye(n)] ])
  d = -10*np.ones(m)

  # Initializing the initial values ----------------------------------------------
  x0 = np.zeros(n)
  s0 = np.ones(m)
  lamb0 = np.ones(m)

  return G, g, C, d, x0, lamb0, s0

In [ ]:
n=3
np.random.seed(3)
G, g, C, d, x0, lamb0, s0 = test_prob_gen(n)
z0 = np.concatenate((x0, lamb0, s0), axis=None)

In [ ]:
-g

array([-1.78862847, -0.43650985, -0.09649747])

In [ ]:
z=z0

In [ ]:
z0, iterations, donnumber = Newton_mod(G,C,d,g,n,z0,boolean=False)

F:  [-1.78862847 -0.43650985 -0.09649747  9.          9.          9.
  9.          9.          9.         -1.         -1.         -1.
 -1.         -1.         -1.        ]
delta_z:  [ -0.59620949  -0.14550328  -0.03216582  -9.40379051  -9.85449672
  -9.96783418 -10.59620949 -10.14550328 -10.03216582   8.40379051
   8.85449672   8.96783418   9.59620949   9.14550328   9.03216582]
alpha:  0.09437337010332078
mu:  1.0
mu_tilda:  0.10293543825906913
sigma:  0.001090673481181769
new rhv:  [-1.78862847e+00 -4.36509851e-01 -9.64974681e-02  9.00000000e+00
  9.00000000e+00  9.00000000e+00  9.00000000e+00  9.00000000e+00
  9.00000000e+00  7.80285761e+01  8.62576995e+01  8.83909747e+01
  1.00684537e+02  9.17868243e+01  8.96132759e+01]
new delta_z:  [-8.14819638 -1.98854487 -0.43959958 77.17677248 79.24624437 79.83057426
 83.53634038 80.79827939 80.17367637  0.85180362  7.01145513  8.56040042
 17.14819638 10.98854487  9.43959958]
new alpha: 1
updated z:  [-7.74078656 -1.88911763 -0.4176196  74.3179

In [ ]:
lamb0_test = np.array([3.71589669,  7.97567504,  8.61081616,  9.73665312, 9.34868057,  8.91320403])
dlamb_test = np.array([-6.94138883, -8.50540188, -8.75390303, -8.76776751, -8.99846353, -8.86296409])
s0_test = np.array([2.15039588,  8.14272437,  9.5514581, 17.74921462, 11.75688613, 10.3481524])
ds_test = np.array([1.86659791,  0.5408219,   0.15871763, -1.76620841, -0.4404324,  -0.05832812])
alpha = Newton_step(lamb0_test,dlamb_test,s0_test,ds_test)
print(alpha)

0.5353246707546853
